In [ ]:
%load_ext autoreload
%autoreload 2

import os

from polsarpro.dev.devtools import parse_psp_parameter_string
import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.dev.io import open_biomass_l1_product
from polsarpro.dev.io import read_psp_bin
from polsarpro.decompositions import freeman
from polsarpro.dev.metrics import summarize_metrics, visualize_errors
import shutil

# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_convert/"

# change to your data paths

product_name = "BIO_S2_SCS__1S_20251216T034800_20251216T034815_T_G01_M01_C02_T017_F289_01_DJQGAN"

# original dataset
input_data = Path("/data/psp/test_files/") / product_name

# S2 inputs generated with C-PSP
input_psp_dir = Path("/data/psp/test_files") / product_name / "polsarpro/polsarpro"

# output files from C
output_test_dir = Path("/data/psp/res/biomass/freeman_c/")
if not os.path.isdir(output_test_dir):
    os.makedirs(output_test_dir)

In [ ]:
os.path.exists(input_psp_dir / "config.txt")

## Run the C-version on some test data

In [ ]:
input_str = f"""id: {input_psp_dir}
od: {output_test_dir}
iodf: S2
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 14781
fnc: 1567
errf: /tmp"""
result = parse_psp_parameter_string(input_str)
os.system(f"freeman_decomposition.exe {result}")

# CPSP does not create another config file in output dir
shutil.copy(input_psp_dir / "config.txt", output_test_dir)

## Load ALOS data and C outputs

In [ ]:
# uncomment to test on S matrix made with SNAP
S = open_biomass_l1_product(input_data)

## Apply the decomposition

In [ ]:
file_out = "/data/psp/res/biomass/test_freeman.nc"
# netcdf writer cannot overwrite
if os.path.isfile(file_out):
    os.remove(file_out)

with ProgressBar():
    freeman(S, boxcar_size=[7, 7]).to_netcdf(file_out)

# Numerical evaluation

In [ ]:
import matplotlib.pyplot as plt
# open python output (comment if using compute)
out_py = xr.open_dataset("/data/psp/res/biomass/test_freeman.nc")
# open C-PolSARPro outputs
out_c = {}
out_names = (("odd", "Freeman_Odd"), ("double", "Freeman_Dbl"), ("volume", "Freeman_Vol"))
for name in out_names:
    file_name = output_test_dir / f"{name[1]}.bin"
    out_c[name[0]] = read_psp_bin(file_name)

df = summarize_metrics(out_py, out_c, short_titles=False, verbose=False)
df

In [ ]:
visualize_errors(out_py=out_py, out_c=out_c)